# Spark (with PySpark)


In this tutorial, you will learn how to use Apache Spark, a framework for large-scale data processing, within a notebook.
Upon completing this lab you will be able to : 
 + Program in Spark with the Python Language
 + Demonstrate how to read and process data using Spark
 + Compare and contrast RDD and Dataframes. 
 + Build a simple machine learning application with Spark.
 
 
 
# Requirements : Installing Spark, PySpark and configuration to run it on jupyter notebooks

According to your OS, you have to follow the following tutorial :

+ [Windows](https://changhsinlee.com/install-pyspark-windows-jupyter/)
+ [Mac Os X](https://jmedium.com/pyspark-in-mac/)
 
At the end of the installation, you should be able to run the following code that is a kind of Hello word in PySpark.

In [1]:
import findspark
findspark.init('C:/Users/louis/Documents/spark/spark-2.3.0-bin-hadoop2.7')

import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

df = spark.sql('''select 'spark' as hello ''')
df.show()

+-----+
|hello|
+-----+
|spark|
+-----+



In [2]:
spark.stop()

## Part 1 : Understanding Spark RDD’s

### WordCount in Spark
In this part, we will write the wordcount in Spark and apply it on the novel Dracula of Bram Stocker (from the Gutemberg project).

First read the [pg345.txt](./SparkData/pg345.txt) file.

In [3]:
import findspark

findspark.init('C:/Users/louis/Documents/spark/spark-2.3.0-bin-hadoop2.7')
import pyspark

sc=pyspark.SparkContext("local[*]","First program in Spark")

In [4]:
data = sc.textFile('./SparkData/pg345.txt')
type(data)

pyspark.rdd.RDD

To see the content, of the file, we need to run the action `collect` on the  RDD `data`

In [5]:
data.collect()

['The Project Gutenberg EBook of Dracula, by Bram Stoker',
 '',
 'This eBook is for the use of anyone anywhere at no cost and with',
 'almost no restrictions whatsoever.  You may copy it, give it away or',
 're-use it under the terms of the Project Gutenberg License included',
 'with this eBook or online at www.gutenberg.org/license',
 '',
 '',
 'Title: Dracula',
 '',
 'Author: Bram Stoker',
 '',
 'Release Date: August 16, 2013 [EBook #345]',
 '',
 'Language: English',
 '',
 '',
 '*** START OF THIS PROJECT GUTENBERG EBOOK DRACULA ***',
 '',
 '',
 '',
 '',
 'Produced by Chuck Greif and the Online Distributed',
 'Proofreading Team at http://www.pgdp.net (This file was',
 'produced from images generously made available by The',
 'Internet Archive)',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '                                DRACULA',
 '',
 '',
 '',
 '',
 '',
 '                                DRACULA',
 '',
 '                                  _by_',
 '',
 '                              Bram Stoke

With the RDD `data`, from the previous cell, execute a `flatMap()` for each line in the input and then convert it to lower case, remove the commas, split the words on a space and store in the RDD `words`

In [6]:
#map step

word_counts = data.flatMap(lambda line: line.split(' ')).map(lambda line:line.replace(",","")).map(lambda line:line.lower())


Execute the command with the action `take()` and retrieve the first 10 words from the `flatMap()`transformation

In [7]:

print(word_counts.take(10))

['the', 'project', 'gutenberg', 'ebook', 'of', 'dracula', 'by', 'bram', 'stoker', '']


Perform a classic `map()` to create a tuple where each word has a count of 1

In [8]:
word_counts=word_counts.map(lambda word: (word,1))
print(word_counts.take(10))

[('the', 1), ('project', 1), ('gutenberg', 1), ('ebook', 1), ('of', 1), ('dracula', 1), ('by', 1), ('bram', 1), ('stoker', 1), ('', 1)]


Write the reducing function

In [9]:
# reducer
word_counts=word_counts.reduceByKey(lambda x,y: x+y) 

print(word_counts.take(10))

[('project', 83), ('gutenberg', 24), ('ebook', 10), ('of', 3719), ('dracula', 29), ('stoker', 5), ('', 9306), ('this', 629), ('is', 1489), ('use', 48)]


Get all words that occur more than once and sort them alphabetically:

In [10]:
print(word_counts.filter(lambda key: key[1]>1).sortByKey().take(20))

[('', 9306), ('"\'my', 2), ('"_17', 4), ('"_2', 2), ('"_24', 2), ('"_25', 4), ('"_6', 2), ('"_czarina', 2), ('"a', 12), ('"ah', 18), ('"aha!"', 2), ('"all', 8), ('"am', 2), ('"and', 53), ('"are', 5), ('"arthur!', 2), ('"as', 6), ('"at', 3), ('"be', 2), ('"because', 8)]


Now, get all words that occur more than once and sort them by frequency. Select the first 20. Hints : look at the `takeOrdered` action.

In [11]:

print(word_counts.filter(lambda key: key[1]>1).takeOrdered(20, lambda y: -1*y[1]))

[('', 9306), ('the', 7984), ('and', 5840), ('i', 4527), ('to', 4518), ('of', 3719), ('a', 2933), ('he', 2523), ('in', 2512), ('that', 2415), ('it', 1876), ('was', 1848), ('as', 1573), ('for', 1514), ('we', 1499), ('is', 1489), ('his', 1457), ('not', 1350), ('with', 1314), ('you', 1302)]


### A simple exercice

+ Create a Python collection of 10,000 integers
+ Create a Spark base RDD from that collection
+ Subtract one from each value using map
+ Perform action collect to view results
+ Perform action count to view counts
+ Apply transformation filter and view results with collect

In [12]:
# create dataset
import random
n= 10000
numLow=-n
numHigh=2*n
setOfNumbers = set()
while len(setOfNumbers) < n:
    setOfNumbers.add(random.randint(numLow, numHigh))

numbers=sc.parallelize(setOfNumbers)

In [13]:
#vizulaise data created
print(numbers.take(10))
numbers=numbers.map(lambda num:num-1)
print(numbers.take(10))
numbers.count()

[0, 6, 8, 10, 12, 17, 18, 22, 24, 26]
[-1, 5, 7, 9, 11, 16, 17, 21, 23, 25]


10000

### An improved WordCount

Print the top 10 most frequent words with their probability of appearance


In [14]:
print(word_counts.filter(lambda key: key[1]>1).takeOrdered(10, lambda y: -1*y[1]))

[('', 9306), ('the', 7984), ('and', 5840), ('i', 4527), ('to', 4518), ('of', 3719), ('a', 2933), ('he', 2523), ('in', 2512), ('that', 2415)]


Get rid of special characters (.,:!?')


In [15]:
clear=word_counts.map(lambda string: ''.join(e for e in string[0] if e.isalnum()))
clear=clear.map(lambda word: (word,1)).reduceByKey(lambda x,y: x+y) 
print(clear.take(20))


[('project', 2), ('gutenberg', 4), ('ebook', 2), ('of', 7), ('dracula', 6), ('stoker', 1), ('', 7), ('this', 11), ('is', 15), ('use', 3), ('anyone', 2), ('anywhere', 3), ('at', 7), ('no', 17), ('restrictions', 1), ('whatsoever', 2), ('may', 11), ('give', 4), ('away', 6), ('reuse', 1)]


Identify the transformations and the actions in your script


In [16]:
print(clear.filter(lambda key: key[1]>1).sortByKey().take(20))

[('', 7), ('1', 7), ('10', 4), ('11', 2), ('12', 3), ('13', 2), ('14', 2), ('16', 2), ('17', 3), ('18', 3), ('19', 2), ('1c', 2), ('1e', 2), ('1e1', 2), ('1e7', 2), ('1e8', 2), ('1f3', 2), ('2', 4), ('20', 4), ('24', 3)]


In [17]:
sc.stop()

## Part 2 : Spark SQL and dataframes

In this part, you will explore Spark DataFrames and the SQL Context. In particular, we will work on a database that contains a sample of the world population by working on data that comes from [pplapi](http://pplapi.com/). The file [agents.json](./SparkData/agents.json) is a file that was extracted from this api using the following command :


`wget https://s3-eu-west-1.amazonaws.com/course.oc-static.com/courses/4297166/agents.json`

In [18]:
import findspark

findspark.init('C:/Users/louis/Documents/spark/spark-2.3.0-bin-hadoop2.7')
import pyspark

sc=pyspark.SparkContext("local[*]","First program in Spark")

import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

df = spark.read.json("./SparkData/agents.json")



Print the 5 first lines of the dataframe `df`

In [19]:
df.show(5)

+------------+----------+------------------+------------------+------+
|country_name|        id|          latitude|         longitude|   sex|
+------------+----------+------------------+------------------+------+
|       China| 227417393| 33.15219798270325|100.85840672174572|  Male|
|       Haiti|6821129477|19.325567983697297|-72.43795260265814|Female|
|       India|2078667700|23.645271492037235| 80.85636526088884|Female|
|       China| 477556555| 33.45864668881662| 93.33604038078953|Female|
|       India|1379059984|28.816938290678692|  80.7728698035823|Female|
+------------+----------+------------------+------------------+------+
only showing top 5 rows



Write the spark instructions that enable to display the number of French agents.

In [20]:
french=df.filter(df['country_name']=="France").count()
print("number of French agents: ",french)

number of French agents:  94


Write the spark instructions that enable to display the number of Indian female agents.

In [21]:
indian=df.filter(df['country_name']=="India")
female=indian.filter(indian['sex']=="Female").count()
print("number of Indian female agents: ",female)

number of Indian female agents:  828


Using the notion of temporary view (function `createTempView`), create a temporary view associated to the dataframe `df`. 

In [22]:
df.createTempView("temp_table")


Write some SQL query on the resulting table.

In [23]:
spark.sql("SELECT country_name,id FROM temp_table ORDER BY id DESC LIMIT 10").show()

+-----------------+----------+
|     country_name|        id|
+-----------------+----------+
| French Polynesia|7170821229|
|       Cabo Verde|7167692449|
|         Suriname|7166451460|
|         Suriname|7166235088|
|            Macau|7166034642|
|       Montenegro|7164357515|
|Equatorial Guinea|7163867872|
|           Bhutan|7163256789|
|           Bhutan|7163004645|
|           Bhutan|7162877973|
+-----------------+----------+



In [25]:

spark.sql("SELECT country_name,count(country_name) as number FROM temp_table GROUP BY country_name ").show()

+------------+------+
|country_name|number|
+------------+------+
|        Chad|    20|
|      Russia|   185|
|    Paraguay|     9|
|       Yemen|    43|
|Korea, North|    47|
|     Senegal|    17|
|      Sweden|    18|
|  Cabo Verde|     1|
|      Guyana|     2|
| Philippines|   160|
|       Burma|    71|
|     Eritrea|     8|
|    Djibouti|     3|
|    Malaysia|    54|
|   Singapore|     8|
|      Turkey|   103|
|      Malawi|    30|
|        Iraq|    46|
|     Germany|   114|
|     Comoros|     2|
+------------+------+
only showing top 20 rows



In [26]:
spark.stop()
sc.stop()

You can also create a dataframe from an existing RDD as shown on the example below

In [27]:
sc=pyspark.SparkContext("local[*]","Dataframe examples in Spark")
sqlContext = pyspark.SQLContext(sc)



In [28]:
documents_rdd = sc.parallelize([
        [1, 'cats are cute', 0],
        [2, 'dogs are playfull', 0],
        [3, 'lions are big', 1],
        [4, 'cars are fast', 1]])
users_rdd = sc.parallelize([
        [0, 'Alice', 20],
        [1, 'Bob', 23],
        [2, 'Charles', 32]])

In [29]:
documents_df = documents_rdd.toDF(['doc_id', 'text', 'user_id'])
users_df = users_rdd.toDF(['user_id', 'name', 'age'])

# printing the inferred schema for documents
documents_df.printSchema()


root
 |-- doc_id: long (nullable = true)
 |-- text: string (nullable = true)
 |-- user_id: long (nullable = true)



Some functions can be apply to a column or to different columns. Here, we compute the age avarage in the dataframe `users_df`. 

In [30]:
from pyspark.sql import functions as fn

user_age_df = users_df.select(fn.avg('age')).show()

+--------+
|avg(age)|
+--------+
|    25.0|
+--------+




Compute the max of age in the dataframe `users_df`. 

In [31]:
user_max_df = users_df.select(fn.max('age'))

Join (as in SQL) the two dataframes `users_df` and `documents_df`

In [32]:
users_df.join(documents_df,"user_id").show()

+-------+-----+---+------+-----------------+
|user_id| name|age|doc_id|             text|
+-------+-----+---+------+-----------------+
|      0|Alice| 20|     1|    cats are cute|
|      0|Alice| 20|     2|dogs are playfull|
|      1|  Bob| 23|     3|    lions are big|
|      1|  Bob| 23|     4|    cars are fast|
+-------+-----+---+------+-----------------+



Outer Join (Left) (as in SQL) the two dataframes `users_df` and `documents_df`

In [33]:
users_df.join(documents_df,"user_id","leftouter").show()

+-------+-------+---+------+-----------------+
|user_id|   name|age|doc_id|             text|
+-------+-------+---+------+-----------------+
|      0|  Alice| 20|     1|    cats are cute|
|      0|  Alice| 20|     2|dogs are playfull|
|      1|    Bob| 23|     3|    lions are big|
|      1|    Bob| 23|     4|    cars are fast|
|      2|Charles| 32|  null|             null|
+-------+-------+---+------+-----------------+



## Part 3 : MLlib - Spark for data analysis and machine learning

In this part, we will see how to create a text classifiation application with Spark. We will use some data from the newsgroups [Usenet](https://en.wikipedia.org/wiki/Usenet_newsgroup) and the objective is to predict the topic of the news.
 
The train data is [here](./SparkData/20ng-train-all-terms.txt) and the test data [here](./SparkData/20ng-test-all-terms.txt).
The are obtained using the following commands: 
`wget http://ana.cachopo.org/datasets-for-single-label-text-categorization/20ng-train-all-terms.txt`
and
`wget http://ana.cachopo.org/datasets-for-single-label-text-categorization/20ng-test-all-terms.txt`

Load these data as some RDDs and translate them in DataFrames.

In [35]:
sc.stop()

findspark.init()
from pyspark.ml.feature import CountVectorizer

sc=pyspark.SparkContext("local[*]","machine learning in Spark")
sqlContext = pyspark.SQLContext(sc)

from pyspark.sql import Row
from pyspark.sql.functions import udf
from pyspark.sql.types import *

def load_dataframe(path):
    data = sc.textFile(path)
    data=data.map(lambda line: line.split('\t'))
    data=data.toDF(["labels","text"])
    return(data)

def my_split(data):
    text=data
    return(text.split(" "))

#load data

train_data = load_dataframe("./SparkData/train-all-terms.txt")
test_data = load_dataframe("./SparkData/test-all-terms.txt")

# create function to split text into a list of strings
splitudf=udf(my_split,ArrayType(StringType()))
train_data=train_data.withColumn('text2', splitudf('text'))
test_data=test_data.withColumn('text2', splitudf('text'))

#size of sets
n_train=train_data.count()
n_test=test_data.count()

#combine sets 
data=train_data.union(test_data)

train_data.groupBy("labels").count().show()
test_data.groupBy("labels").count().show()

+--------------------+-----+
|              labels|count|
+--------------------+-----+
|      comp.windows.x|  593|
|        misc.forsale|  585|
|    rec.sport.hockey|  600|
|  rec.sport.baseball|  597|
|  talk.politics.guns|  545|
|comp.os.ms-window...|  572|
|  talk.politics.misc|  465|
|comp.sys.ibm.pc.h...|  590|
|       comp.graphics|  584|
|soc.religion.chri...|  598|
|comp.sys.mac.hard...|  578|
|  talk.religion.misc|  377|
|talk.politics.mid...|  564|
|     rec.motorcycles|  598|
|           rec.autos|  594|
|         alt.atheism|  480|
|     sci.electronics|  591|
|           sci.space|  593|
|             sci.med|  594|
|           sci.crypt|  595|
+--------------------+-----+

+--------------------+-----+
|              labels|count|
+--------------------+-----+
|      comp.windows.x|  392|
|        misc.forsale|  390|
|    rec.sport.hockey|  399|
|  rec.sport.baseball|  397|
|  talk.politics.guns|  364|
|comp.os.ms-window...|  394|
|  talk.politics.misc|  310|
|comp.sys.ibm

A first step is to represent our data, i.e. the messages in the form of a bag-of-word representation using the spark method `CountVectorizer` documented [here](https://spark.apache.org/docs/2.1.0/ml-features.html#countvectorizer)

In [36]:
# bag of word representation of test and train data

cv = CountVectorizer(inputCol="text2", outputCol="features")
model = cv.fit(train_data)
model=cv.fit(train_data)

result = model.transform(train_data)
result.show(truncate=True)



+-----------+--------------------+--------------------+--------------------+
|     labels|                text|               text2|            features|
+-----------+--------------------+--------------------+--------------------+
|alt.atheism|alt atheism faq a...|[alt, atheism, fa...|(73713,[0,1,2,3,4...|
|alt.atheism|alt atheism faq i...|[alt, atheism, fa...|(73713,[0,1,2,3,4...|
|alt.atheism|re gospel dating ...|[re, gospel, dati...|(73713,[0,1,2,3,4...|
|alt.atheism|re university vio...|[re, university, ...|(73713,[0,1,2,3,4...|
|alt.atheism|re soc motss et a...|[re, soc, motss, ...|(73713,[0,1,2,3,4...|
|alt.atheism|re a visit from t...|[re, a, visit, fr...|(73713,[0,1,2,3,4...|
|alt.atheism|re political athe...|[re, political, a...|(73713,[0,1,3,4,6...|
|alt.atheism|re an anecdote ab...|[re, an, anecdote...|(73713,[0,1,2,3,4...|
|alt.atheism|re political athe...|[re, political, a...|(73713,[0,1,2,3,4...|
|alt.atheism|re pompous ass km...|[re, pompous, ass...|(73713,[0,4,5,11,...|

Displaying of the distinct labels in the datasets

In [37]:
train_data.select("labels").distinct().sort("labels").show(truncate=False)

+------------------------+
|labels                  |
+------------------------+
|alt.atheism             |
|comp.graphics           |
|comp.os.ms-windows.misc |
|comp.sys.ibm.pc.hardware|
|comp.sys.mac.hardware   |
|comp.windows.x          |
|misc.forsale            |
|rec.autos               |
|rec.motorcycles         |
|rec.sport.baseball      |
|rec.sport.hockey        |
|sci.crypt               |
|sci.electronics         |
|sci.med                 |
|sci.space               |
|soc.religion.christian  |
|talk.politics.guns      |
|talk.politics.mideast   |
|talk.politics.misc      |
|talk.religion.misc      |
+------------------------+



We will now apply a [NaiveBayes](https://spark.apache.org/docs/latest/ml-classification-regression.html#naive-bayes) classifier to our problem. 
Take the time to read the doc and apply it to our problem. You will first have to associate a number to each label. You can use the [`String Indexer`](https://spark.apache.org/docs/2.1.0/ml-features.html#stringindexer) function of pyspark for that.


In [38]:
# Transformation of the label into a number
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="labels", outputCol="label")
indexed_train = indexer.fit(result).transform(result)

#transformation of the data
indexed_train.show()

#summary
indexed_train.select("labels","label").distinct().sort("label").show(truncate=False)

+-----------+--------------------+--------------------+--------------------+-----+
|     labels|                text|               text2|            features|label|
+-----------+--------------------+--------------------+--------------------+-----+
|alt.atheism|alt atheism faq a...|[alt, atheism, fa...|(73713,[0,1,2,3,4...| 17.0|
|alt.atheism|alt atheism faq i...|[alt, atheism, fa...|(73713,[0,1,2,3,4...| 17.0|
|alt.atheism|re gospel dating ...|[re, gospel, dati...|(73713,[0,1,2,3,4...| 17.0|
|alt.atheism|re university vio...|[re, university, ...|(73713,[0,1,2,3,4...| 17.0|
|alt.atheism|re soc motss et a...|[re, soc, motss, ...|(73713,[0,1,2,3,4...| 17.0|
|alt.atheism|re a visit from t...|[re, a, visit, fr...|(73713,[0,1,2,3,4...| 17.0|
|alt.atheism|re political athe...|[re, political, a...|(73713,[0,1,3,4,6...| 17.0|
|alt.atheism|re an anecdote ab...|[re, an, anecdote...|(73713,[0,1,2,3,4...| 17.0|
|alt.atheism|re political athe...|[re, political, a...|(73713,[0,1,2,3,4...| 17.0|
|alt

In [39]:
#  Application of the NaivesBayes models
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Split set into training and test set 70/30%
splits = indexed_train.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1]


# create the trainer and set its parameters
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")

# train the model
model = nb.fit(train)



In [41]:
# T Application of the trained model to the test dataset and displaying the prediction compared to the real value for the 20 first messages
# select example rows to display.
predictions = model.transform(test)
predictions.show()

# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))


+-----------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
|     labels|                text|               text2|            features|label|       rawPrediction|         probability|prediction|
+-----------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
|alt.atheism|a fundamental con...|[a, fundamental, ...|(73713,[0,1,2,3,4...| 17.0|[-2296.4551153397...|[2.34316728465042...|       1.0|
|alt.atheism|a remarkable admi...|[a, remarkable, a...|(73713,[0,1,2,3,4...| 17.0|[-1346.4551682889...|[9.33427206729450...|      17.0|
|alt.atheism|a word of advice ...|[a, word, of, adv...|(73713,[0,2,3,4,5...| 17.0|[-745.83915899261...|[4.69420223601775...|      17.0|
|alt.atheism|cannanite genocid...|[cannanite, genoc...|(73713,[0,5,6,8,9...| 17.0|[-332.04696401390...|[4.21721566864230...|       1.0|
|alt.atheism|islam vs the jeho...|[islam, vs, th